In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
current_pwd = os.getcwd()

possible_paths = [
    '/home/export/soheuny/SRFinder/soheun/notebooks', 
    '/home/soheuny/HH4bsim/soheun/notebooks'
]
    
assert os.getcwd() in possible_paths, f"Did you change the path? It should be one of {possible_paths}"
os.chdir("..")

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import torch

from plots import hist_events_by_labels
from events_data import EventsData
from fvt_classifier import FvTClassifier
from tst_info import TSTInfo
# import LogNorm
from matplotlib.colors import LogNorm


features = [
    "sym_Jet0_pt", "sym_Jet1_pt", "sym_Jet2_pt", "sym_Jet3_pt",
    "sym_Jet0_eta", "sym_Jet1_eta", "sym_Jet2_eta", "sym_Jet3_eta",
    "sym_Jet0_phi", "sym_Jet1_phi", "sym_Jet2_phi", "sym_Jet3_phi",  
    "sym_Jet0_m", "sym_Jet1_m", "sym_Jet2_m", "sym_Jet3_m",
]

In [2]:
from itertools import product
from training_info import TrainingInfo
from plots import calibration_plot, plot_rewighted_samples_by_model
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from ancillary_features import get_m4j
from pl_callbacks import CalibrationPlotCallback, ReweightedPlotCallback

# use tex
plt.rcParams["text.usetex"] = True
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "Times New Roman"

plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.titlesize"] = 20
plt.rcParams["axes.titlesize"] = 20
plt.rcParams["axes.labelsize"] = 15
plt.rcParams["figure.labelsize"] = 20
plt.rcParams["lines.markersize"] = 3

In [3]:
def smear_features(X: np.ndarray, noise_scale: float, seed: int, hard_cutoff: bool = False, scale_mode: str = "std"):
    if scale_mode == "std":
        base_scale = np.std(X, axis=0)
    elif scale_mode == "range":
        features_min = np.min(X, axis=0)
        features_max = np.max(X, axis=0)
        base_scale = features_max - features_min
    else:
        raise ValueError(f"Invalid scale_mode: {scale_mode}")
    
    accept_mask = np.zeros_like(X, dtype=bool)
    X_smeared = np.zeros_like(X)
    
    np.random.seed(seed)
    if hard_cutoff:
        while True:
            X_smeared[~accept_mask] = (X + noise_scale * base_scale * np.random.randn(*X.shape))[~accept_mask]
            accept_mask = (X_smeared >= features_min) & (X_smeared <= features_max)
            if np.all(accept_mask):
                break
    else:
        X_smeared = X + noise_scale * base_scale * np.random.randn(*X.shape)
        
    return X_smeared

In [4]:
import time
import pytorch_lightning as pl
from events_data import events_from_scdinfo
from training_info import TrainingInfo

n_3b = 100_0000
device = torch.device("cuda")
experiment_name = "better_base_fvt_training"
signal_filename = "HH4b_picoAOD.h5"
ratio_4b = 0.5

seeds = [0]
hparam_filter = {
    "experiment_name": lambda x: x in [experiment_name],
    "dataset": lambda x: all([x["seed"] in seeds, 
                              x["n_3b"] == n_3b, 
                              x["signal_ratio"] == 0.02]),
}
hashes = TrainingInfo.find(hparam_filter)

noise_scale = 0.5
depth = 8
batch_size = 1024
max_epochs = 30

gamma_dict = {}

for tinfo_hash in hashes:
    tinfo = TrainingInfo.load(tinfo_hash)
    model = tinfo.load_trained_model("best")
    seed = tinfo.hparams["dataset"]["seed"]
    n_3b = tinfo.hparams["dataset"]["n_3b"]
    signal_ratio = tinfo.hparams["dataset"]["signal_ratio"]
    print(
        f"seed={seed}, n_3b={n_3b}, signal_ratio={signal_ratio}"
    )
    
    model.eval()
    model.to(device)
    events_train = events_from_scdinfo(tinfo.scdinfo, features, signal_filename)
    events_train.shuffle(seed=seed)
    q_repr, view_scores = model.representations(events_train.X_torch)
    
    
    from attention_classifier import AttentionClassifier
    from sklearn.model_selection import train_test_split
    from torch.utils.data import TensorDataset

    
    tb_log_dir = "smeared_fvt_training_tmp"
    optimizer_config = {
        "type": "Adam",
        "lr": 1e-2,
    }
    lr_scheduler_config = {
        "type": "ReduceLROnPlateau",
        "factor": 0.25,
        "patience": 3,
        "min_lr": 2e-4,
        "cooldown": 1,
        "threshold": 1e-4,
    }
    early_stop_patience = None
    dataloader_config = {
        "batch_size": batch_size,
        "batch_size_milestones": [1, 3, 6, 10, 15],
        "batch_size_multiplier": 2,
    }
    
    from dataset import MotherSamples
    ms_scdinfo = MotherSamples.load(tinfo._ms_hash).scdinfo
    events_tst = events_from_scdinfo(ms_scdinfo[~tinfo._ms_idx], features, signal_filename)
    events_tst.shuffle(seed=seed)
    probs_4b_tst = model.predict(events_tst.X_torch, do_tqdm=True)[:, 1].to("cpu").numpy()
    q_repr_tst, _ = model.representations(events_tst.X_torch)
    gamma_tst = probs_4b_tst / (1 - probs_4b_tst)
    gamma_dict[0.0] = gamma_tst

    
    for noise_scale in [0.25, 0.5, 1.0, 1.5, 2.0, 3.0, 4.0]:
        q_repr_smear = smear_features(q_repr.numpy(), noise_scale, seed, hard_cutoff=False, scale_mode="std")
        q_sm_train, q_sm_val, is_4b_train, is_4b_val, weights_train, weights_val = train_test_split(
        q_repr_smear, events_train.is_4b, events_train.weights, test_size=0.33, random_state=seed
        )
        q_sm_train = torch.tensor(q_sm_train, dtype=torch.float32)
        q_sm_val = torch.tensor(q_sm_val, dtype=torch.float32)
        is_4b_train = torch.tensor(is_4b_train, dtype=torch.long)
        is_4b_val = torch.tensor(is_4b_val, dtype=torch.long)
        weights_train = torch.tensor(weights_train, dtype=torch.float32)
        weights_val = torch.tensor(weights_val, dtype=torch.float32)


        q_sm_train = q_sm_train[:batch_size * (len(q_sm_train) // batch_size)]
        is_4b_train = is_4b_train[:batch_size * (len(is_4b_train) // batch_size)]
        weights_train = weights_train[:batch_size * (len(weights_train) // batch_size)]

        q_sm_val = q_sm_val[:batch_size * (len(q_sm_val) // batch_size)]
        is_4b_val = is_4b_val[:batch_size * (len(is_4b_val) // batch_size)]
        weights_val = weights_val[:batch_size * (len(weights_val) // batch_size)]

        train_dataset = TensorDataset(q_sm_train, is_4b_train, weights_train)
        val_dataset = TensorDataset(q_sm_val, is_4b_val, weights_val)
        
        pl.seed_everything(seed)
        att_classifier = AttentionClassifier(
            dim_q=q_repr.shape[1],
            num_classes=2,
            depth=depth,
            run_name=f"smeared_fvt_training_tmp_depth={depth}_seed={seed}_lrs=True_bsch=True_noise_scale={noise_scale}",
        )

        att_classifier.to(device)
        att_classifier.fit(
            train_dataset, val_dataset,
            max_epochs=max_epochs,
            optimizer_config=optimizer_config,
            lr_scheduler_config=lr_scheduler_config,
            early_stop_patience=early_stop_patience,
            dataloader_config=dataloader_config,
            tb_log_dir=tb_log_dir,
        )
        
        att_classifier.eval()
        att_classifier.to(device)
        probs_4b_smeared_tst = att_classifier.predict(q_repr_tst, do_tqdm=True)[:, 1].to("cpu").numpy()

        gamma_tilde_tst = probs_4b_smeared_tst / (1 - probs_4b_smeared_tst)

        from plots import plot_sr_stats

        fig, ax = plt.subplots(1, 1, figsize=(6, 6))
        fig.suptitle(f"Noise Scale: {noise_scale}")
        plot_sr_stats(events_tst, gamma_tst, ax, label="Original")
        plot_sr_stats(events_tst, gamma_tilde_tst, ax, label="Smeared")
        plot_sr_stats(events_tst, gamma_tst / gamma_tilde_tst, ax, label="Original / Smeared")
        plt.legend()
        plt.show()
        
        gamma_dict[noise_scale] = gamma_tilde_tst

seed=0, n_3b=1000000, signal_ratio=0.02


100%|██████████| 64/64 [00:03<00:00, 16.37it/s]
[rank: 0] Seed set to 0
/home/export/soheuny/.conda/envs/coffea_torch/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/export/soheuny/.conda/envs/coffea_torch/lib/py ...
[n02:3335822] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.n02.3393/jf.0/1997275136/shared_mem_cuda_pool.n02 could be created.
[n02:3335822] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/export/soheuny/.conda/envs/coffea_torch/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command i

Epoch 10:   0%|          | 0/43 [00:00<?, ?it/s, v_num=2, val_loss=0.657, val_sigma_sq=1.970, train_loss=0.658]        

/home/export/soheuny/.conda/envs/coffea_torch/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (43) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 15:   0%|          | 0/22 [00:00<?, ?it/s, v_num=2, val_loss=0.657, val_sigma_sq=1.350, train_loss=0.658]         

/home/export/soheuny/.conda/envs/coffea_torch/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (22) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 

Error in callback <bound method AutoreloadMagics.post_execute_hook of <IPython.extensions.autoreload.AutoreloadMagics object at 0x7fa72fb55f50>> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 